In [1]:
import torch
import math
from random import randint
from multipledispatch import dispatch

class Symb:
    
    def __init__(self, copy = None):
        if copy is None:
            self.non_copy_constructor()
        else:
            self.copy_constructor(copy)
    
    def non_copy_constructor(self):
        self.sym = toascii(randint(0, 9))
    
    def copy_constructor(self, copy):
        self.sym = copy.sym
        
    def show_symb(self):
        for r in self.sym:
            for c in r:
                print(c, end="")
            print()
            
    def mk_noise(self,p):
        pp = torch.empty(5,6).uniform_(p,p)
        self.sym = toascii(torch.bernoulli(pp))
    
    def add_noise(self,noise):
        res=torch.logical_xor(totensor(symb2.sym),totensor(noise.sym))
        self.sym=toascii(res.float())
        
    def bad_recog(self, p):
        cur=1
        cur_p=1
        cur_i=10
        cur_ip=10
        a=totensor(self.sym)
        for i in range(0, 10):
            xo=torch.logical_xor(a, totensor(toascii(i)))
            an=torch.logical_and(xo.float(), totensor(toascii(i)))
            r=torch.sum(an.float())/30
            print("now: ",i,"and r=",r,"and abs(r-p)=", abs(r-p))
            if (r< cur):
                cur = r
                cur_i=i
                print(cur," for rec: ",cur_i)
            elif(r==cur):
                print("maybe ",i)
            if (abs(r-p) < cur_p):
                cur_p=abs(r-p)
                cur_ip=i
                print(cur_p," (p) for rec: ",cur_ip)
            elif(abs(r-p)==cur_p):
                print("(p) maybe ",i)
            print()
        print("result: ", cur_i, "and", cur_ip)
        print()
        self.sym=toascii(cur_i)
        self.show_symb()
        print()
        self.sym=toascii(cur_ip)
        self.show_symb()
        
    def recog(self, p):
        cur_i=10
        a=totensor(self.sym)
        one=totensor(toascii(-1))
        for i in range(0, 10):
            x1=torch.logical_xor(a, totensor(toascii(i)))
            x2=torch.logical_xor(one, x1.float())
            cur_res=torch.sum(x1.float())*math.log(p)+torch.sum(x2.float())*math.log(1-p)
            print(cur_res," and ", i)
            if (i==0):
                cur_i=i
                res=cur_res
            elif (cur_res>res):
                cur_i=i
                res=cur_res
                print("NEW i ", i)
        print("result: ", cur_i)
        self.sym=toascii(a)
    
@dispatch(int)
def toascii(num):
    likeswitch = {
        
        0: [['-','0','0','0','0','-'],
            ['0','0','-','-','0','0'],
            ['0','0','-','-','0','0'],
            ['0','0','-','-','0','0'],
            ['-','0','0','0','0','-']],
        
        1: [['-','-','0','0','-','-'],
            ['-','0','0','0','-','-'],
            ['-','-','0','0','-','-'],
            ['-','-','0','0','-','-'],
            ['-','0','0','0','0','-']],
        
        2: [['-','0','0','0','0','-'],
            ['0','0','-','-','0','0'],
            ['-','-','-','0','0','-'],
            ['-','-','0','0','-','-'],
            ['0','0','0','0','0','0']],
        
        3: [['-','0','0','0','0','-'],
            ['0','0','-','-','0','0'],
            ['-','-','-','0','0','-'],
            ['0','0','-','-','0','0'],
            ['-','0','0','0','0','-']],
        
        4: [['0','0','-','-','0','0'],
            ['0','0','-','-','0','0'],
            ['0','0','0','0','0','0'],
            ['-','-','-','-','0','0'],
            ['-','-','-','-','0','0']],
        
        5: [['0','0','0','0','0','0'],
            ['0','0','-','-','-','-'],
            ['0','0','0','0','0','-'],
            ['-','-','-','-','0','0'],
            ['0','0','0','0','0','-']],
        
        6: [['-','0','0','0','0','0'],
            ['0','0','-','-','-','-'],
            ['0','0','0','0','0','-'],
            ['0','0','-','-','0','0'],
            ['-','0','0','0','0','-']],
        
        7: [['0','0','0','0','0','0'],
            ['-','-','-','0','0','-'],
            ['-','-','0','0','-','-'],
            ['-','0','0','-','-','-'],
            ['0','0','-','-','-','-']],
        
        8:  [['-','0','0','0','0','-'],
            ['0','0','-','-','0','0'],
            ['-','0','0','0','0','-'],
            ['0','0','-','-','0','0'],
            ['-','0','0','0','0','-']],
        
        9:  [['-','0','0','0','0','-'],
            ['0','0','-','-','0','0'],
            ['-','0','0','0','0','0'],
            ['-','-','-','-','0','0'],
            ['-','0','0','0','0','-']],
        
        -1: [['0','0','0','0','0','0'],
            ['0','0','0','0','0','0'],
            ['0','0','0','0','0','0'],
            ['0','0','0','0','0','0'],
            ['0','0','0','0','0','0']]
    }
    return likeswitch.get(num, "error")

@dispatch(torch.Tensor)
def toascii(t):
    t=t.tolist()
    for i in range(len(t)):
        for j in range(len(t[i])):
            if t[i][j]== 1.0:
                t[i][j]='0'
            else:
                t[i][j]='-'
    return t

@dispatch(list)
def totensor(l):
    for i in range(len(l)):
        for j in range(len(l[i])):
            if l[i][j]=='0':
                l[i][j]=1.0
            else:
                l[i][j]=0.0
    return torch.Tensor(l)

In [25]:
symb1=Symb()
symb1.show_symb()
print()
noise=Symb()
noise.mk_noise(0.1)
noise.show_symb()
print()
symb2=Symb(symb1)
symb2.add_noise(noise)
symb2.show_symb()
print()
symb2.recog(0.1)

-0000-
00--00
---00-
--00--
000000

0-----
-0----
-0----
------
------

00000-
0---00
-0-00-
--00--
000000

tensor(-31.7247)  and  0
tensor(-33.9220)  and  1
tensor(-9.7525)  and  2
NEW i  2
tensor(-27.3303)  and  3
tensor(-36.1192)  and  4
tensor(-27.3303)  and  5
tensor(-36.1192)  and  6
tensor(-31.7247)  and  7
tensor(-27.3303)  and  8
tensor(-25.1331)  and  9
result:  2
